# ========================================
# 8.smolvla_parol6.ipynb - SmolVLA策略部署与测试
# ========================================
# 
# 📘 **笔记本功能说明**
# 本笔记本用于部署和测试已训练好的SmolVLA（Small Vision-Language-Action）模型
# 
# 🎯 **什么是SmolVLA？**
# SmolVLA是一个小型的视觉-语言-动作模型，它能够：
# - 接收图像输入（摄像头画面）
# - 理解自然语言指令（如"拿起杯子"）
# - 输出机器人动作（关节角度、夹爪状态）
#
# 🔄 **工作流程**
# 1. 加载训练好的模型权重
# 2. 在MuJoCo仿真环境中创建任务场景
# 3. 让模型控制机器人执行任务
# 4. 统计成功率和性能指标
#
# 📊 **评估指标**
# - 成功率：完成任务的比例
# - 平均步数：完成任务所需的控制步数
# - 平均耗时：完成任务所需的时间
#
# ========================================

<img src="./media/rollout3.gif" width="480" height="360">

Deploy trained policy in simulation.

In [ ]:
# ========================================
# Cell 1 - 环境变量设置（必须第一个运行！）
# ========================================
#
# ⚠️  **重要提示**
# 这个cell必须在所有其他cell之前运行，因为它设置了MuJoCo渲染所需的环境变量
# 如果不先运行这个cell，后续的仿真渲染可能会失败
#
# 📝 **环境变量说明**

import os

# ────────────────────────────────────────
# 1. 设置X11显示服务器（用于图形界面）
# ────────────────────────────────────────
# DISPLAY: 指定X服务器的地址（:0表示本地第一个显示器）
# XAUTHORITY: X服务器的认证文件路径（用于权限验证）
os.environ['DISPLAY'] = ':0'
os.environ['XAUTHORITY'] = os.path.expanduser('~/.Xauthority')
print(f"✓ DISPLAY设置为: {os.environ['DISPLAY']}")

# ────────────────────────────────────────
# 2. 强制使用GPU渲染（关键性能设置！）
# ────────────────────────────────────────
# MUJOCO_GL: 指定OpenGL渲染后端
#   - 'egl': 使用EGL后端，支持无头（headless）GPU加速渲染
#   - 'glfw': 使用GLFW后端，需要显示器（较慢）
#   - 'osmesa': 使用软件渲染（最慢）
# 
# 💡 为什么选择'egl'？
# - 支持GPU硬件加速，渲染速度快
# - 支持无头渲染，不需要实际的显示器
# - 适合服务器和云环境
os.environ['MUJOCO_GL'] = 'egl'
print(f"✓ MUJOCO_GL: egl (GPU硬件加速)")

# ────────────────────────────────────────
# 3. NVIDIA GPU渲染优化
# ────────────────────────────────────────
# __GL_SYNC_TO_VBLANK: 垂直同步设置
#   - '0': 关闭垂直同步，提高帧率（不等待显示器刷新）
#   - '1': 开启垂直同步，限制帧率为显示器刷新率
# 
# __GL_YIELD: CPU等待策略
#   - 'NOTHING': GPU工作时CPU不等待，减少延迟
#   - 'USLEEP': GPU工作时CPU休眠，节省资源
os.environ['__GL_SYNC_TO_VBLANK'] = '0'
os.environ['__GL_YIELD'] = 'NOTHING'
print("✓ NVIDIA GPU优化已启用")

# ────────────────────────────────────────
# 4. OpenGL性能优化
# ────────────────────────────────────────
# __GL_FSAA_MODE: 抗锯齿模式
#   - '0': 关闭抗锯齿（提高性能，降低画质）
#   - 其他值: 不同级别的抗锯齿
# 
# __GL_LOG_MAX_ANISO: 各向异性过滤
#   - '0': 关闭各向异性过滤（提高性能）
#   - 其他值: 开启各向异性过滤（提高纹理质量）
os.environ['__GL_FSAA_MODE'] = '0'
os.environ['__GL_LOG_MAX_ANISO'] = '0'
print("✓ OpenGL性能优化已启用")

# ────────────────────────────────────────
# ✅ 设置完成
# ────────────────────────────────────────
print("\n🎉 所有环境变量设置完成！现在可以运行后续的cell了。")

In [ ]:
# ========================================
# Cell 2 - 安装必要的Python依赖包
# ========================================
#
# 📦 **依赖包说明**
#
# 1. transformers==4.50.3
#    - Hugging Face的Transformers库，用于加载和运行预训练的深度学习模型
#    - SmolVLA模型基于Transformer架构构建
#    - 版本4.50.3是经过测试兼容的稳定版本
#
# 2. num2words
#    - 数字转文字库（例如：123 -> "one hundred twenty-three"）
#    - 用于处理自然语言指令中的数字
#
# 3. accelerate
#    - Hugging Face的分布式训练和推理加速库
#    - 自动处理GPU/CPU设备管理
#    - 支持混合精度计算（FP16/BF16），提高推理速度
#
# 4. safetensors>=0.4.3
#    - 安全的模型权重存储格式
#    - 比PyTorch的.pth格式更安全、更快
#    - 防止恶意代码注入
#
# ⏱️  预计安装时间：1-3分钟（取决于网络速度）

!pip install transformers==4.50.3
!pip install num2words
!pip install accelerate
!pip install safetensors>=0.4.3

print("\n✅ 所有依赖包安装完成！")

# ========================================
# 步骤1：下载数据集（可选）
# ========================================
#
# 💡 **这个步骤是可选的**
# 
# 📚 **什么时候需要下载数据集？**
# - 如果你想查看训练数据的内容
# - 如果你想重新训练模型
# - 如果你想分析数据集的统计信息
#
# 📚 **什么时候不需要下载？**
# - 如果你只是想测试已训练好的模型 → 跳过这个cell
# - 如果你已经下载过数据集 → 跳过这个cell
#
# 📦 **数据集内容**
# - 机器人执行抓取和放置任务的演示数据
# - 包含：图像、关节状态、动作、任务描述等
# - 数据格式：LeRobot格式（兼容Hugging Face Datasets）

### [Optional] Download Dataset

In [ ]:
# ========================================
# 方法1：使用git clone下载数据集（已注释）
# ========================================
#
# 📝 **这段代码的作用**
# 如果你想使用已收集的数据集，可以从Hugging Face下载
#
# 🔧 **使用方法**
# 取消下面这行代码的注释（删除开头的#），然后运行
#
# ⚠️  **注意**
# - 这个方法会下载整个git仓库，包含完整的版本历史
# - 下载速度可能较慢
# - 推荐使用下面一个cell的huggingface-cli方法（更快）

'''
If you want to use the collected dataset, please download it from Hugging Face.
如果想使用已收集的数据集，请从Hugging Face下载。
'''
#!git clone https://huggingface.co/datasets/Jeongeun/omy_pnp_language

In [ ]:
# ========================================
# 方法2：使用huggingface-cli下载数据集（推荐）
# ========================================
#
# 🚀 **推荐使用这个方法！**
#
# 📝 **命令解释**
# huggingface-cli download \
#   Jeongeun/omy_pnp_language \    # Hugging Face上的数据集ID
#   --repo-type dataset \           # 指定仓库类型为数据集（而不是模型）
#   --local-dir ./demo_data_language # 下载到本地的目录
#
# 💡 **优点**
# - 只下载数据文件，不包含git历史（更快、更小）
# - 支持断点续传
# - 自动验证文件完整性
#
# 📊 **数据集大小**
# - 约几百MB到几GB（取决于演示数量）
# 
# ⏱️  **预计下载时间**
# - 快速网络：5-15分钟
# - 慢速网络：30-60分钟
#
# 📁 **下载位置**
# 数据将保存在：./demo_data_language/

!huggingface-cli download Jeongeun/omy_pnp_language --repo-type dataset --local-dir ./demo_data_language

print("\n✅ 数据集下载完成！")

# ========================================
# 步骤2：训练模型（可选）
# ========================================
#
# 💡 **这个步骤是可选的**
#
# 🎓 **什么时候需要训练？**
# - 你有自己收集的数据集
# - 你想从头开始训练一个新模型
# - 你想在预训练模型基础上继续微调
#
# 🎓 **什么时候不需要训练？**
# - 你只想测试已训练好的模型 → 跳过这个步骤
# - 你想使用checkpoint目录下已有的模型 → 跳过这个步骤
#
# ⏱️  **训练时间估计**
# - 取决于数据集大小和GPU性能
# - RTX 4060: 约几小时到十几小时
# - 训练会保存多个checkpoint（如2000步、10000步等）
#
# 📝 **配置文件**
# - smolvla_omy.yaml: 包含训练参数、模型架构、数据路径等配置

## Step 2. Train Model

In [ ]:
# ========================================
# 启动训练（带监控）
# ========================================
#
# 📝 **代码说明**
#
# 方法1（已注释）：直接使用命令行训练
# !python train_model.py --config_path smolvla_omy.yaml
#   - 简单直接
#   - 无法在notebook中实时查看训练进度
#
# 方法2（当前使用）：使用train_with_monitor在notebook中训练
# train_with_monitor("smolvla_omy.yaml")
#   - 可以在notebook中实时查看训练进度
#   - 可以看到loss曲线、学习率变化等
#   - 更适合调试和监控
#
# 📊 **训练过程中会看到**
# - 训练loss（损失值，越低越好）
# - 学习率变化
# - 训练步数和时间
# - 定期保存的checkpoint
#
# 💾 **模型保存位置**
# ckpt/smolvla_omy4060_YYYYMMDD_HHMMSS/checkpoints/
#   ├── 002000/  ← 2000步的checkpoint
#   ├── 010000/  ← 10000步的checkpoint
#   └── last/    ← 最后一个checkpoint
#
# ⚠️  **注意**
# - 训练需要大量GPU显存（建议8GB以上）
# - 训练过程可能需要几个小时，可以中途停止
# - 停止后可以从checkpoint继续训练

#!python train_model.py --config_path smolvla_omy.yaml

from train_with_monitor_cell import train_with_monitor
train_with_monitor("smolvla_omy.yaml")

# ========================================
# 步骤3：部署模型
# ========================================
#
# 🚀 **这是最重要的部分！**
#
# 📋 **这个步骤要做什么？**
# 1. 加载训练好的模型权重
# 2. 配置模型的输入输出格式
# 3. 在仿真环境中测试模型性能
# 4. 统计成功率和评估指标
#
# 🔄 **流程概览**
# ┌─────────────────┐
# │ 加载数据集元数据 │ ← 获取输入输出特征定义
# └────────┬────────┘
#          ↓
# ┌─────────────────┐
# │ 创建模型配置    │ ← 设置chunk_size、action_steps等
# └────────┬────────┘
#          ↓
# ┌─────────────────┐
# │ 加载训练好的模型 │ ← 从checkpoint加载权重
# └────────┬────────┘
#          ↓
# ┌─────────────────┐
# │ 创建仿真环境    │ ← 设置任务场景
# └────────┬────────┘
#          ↓
# ┌─────────────────┐
# │ 运行评估循环    │ ← 测试多个回合，统计成功率
# └─────────────────┘

## Step 3. Deploy

In [ ]:
# ========================================
# 导入必要的库
# ========================================
#
# 📚 **库功能说明**
#
# ┌─────────────────────────────────────────────────────────────┐
# │ 数据集相关（LeRobot）                                        │
# └─────────────────────────────────────────────────────────────┘
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset, LeRobotDatasetMetadata
# - LeRobotDataset: 加载和处理机器人数据集
# - LeRobotDatasetMetadata: 数据集元数据（输入输出定义、统计信息等）

from lerobot.common.datasets.utils import write_json, serialize_dict
# - write_json: 保存JSON文件
# - serialize_dict: 将字典序列化为可保存的格式

from lerobot.common.datasets.utils import dataset_to_policy_features
# - 将数据集特征转换为策略可用的格式

from lerobot.common.datasets.factory import resolve_delta_timestamps
# - 解析时间戳差值（用于时序数据）

# ┌─────────────────────────────────────────────────────────────┐
# │ SmolVLA模型相关                                              │
# └─────────────────────────────────────────────────────────────┘
from lerobot.common.policies.smolvla.configuration_smolvla import SmolVLAConfig
# - SmolVLA模型的配置类（定义模型架构、超参数等）

from lerobot.common.policies.smolvla.modeling_smolvla import SmolVLAPolicy
# - SmolVLA策略模型（实际的神经网络）

from lerobot.configs.types import FeatureType
# - 特征类型定义（ACTION, STATE, IMAGE等）

# ┌─────────────────────────────────────────────────────────────┐
# │ 基础库                                                       │
# └─────────────────────────────────────────────────────────────┘
import numpy as np          # 数值计算
import torch                # PyTorch深度学习框架
from PIL import Image       # 图像处理
import torchvision          # 视觉相关工具

print("✅ 所有库导入成功！")

In [ ]:
# ========================================
# 设置计算设备
# ========================================
#
# 🖥️  **设备选择**
#
# device = 'cuda' 表示使用NVIDIA GPU进行推理
#
# 💡 **为什么选择GPU？**
# - GPU并行计算能力强，推理速度快（比CPU快10-100倍）
# - SmolVLA模型较大，CPU推理会非常慢
# - 实时控制需要快速推理（20Hz = 每秒20次）
#
# 🔧 **其他选项**
# device = 'cpu'   # 使用CPU（不推荐，太慢）
# device = 'mps'   # 使用Apple Silicon GPU（Mac M1/M2）
# device = 'cuda:0' # 使用第一块GPU
# device = 'cuda:1' # 使用第二块GPU
#
# ✅ **检查GPU是否可用**
# 运行这个cell后，如果有GPU，应该不会报错
# 如果提示CUDA不可用，说明：
# - 没有NVIDIA GPU
# - 或者PyTorch没有正确安装CUDA版本

device = 'cuda' 

# 验证设备
import torch
if torch.cuda.is_available():
    print(f"✅ GPU可用: {torch.cuda.get_device_name(0)}")
    print(f"   显存大小: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️  警告: GPU不可用，将使用CPU（推理速度会很慢）")
    device = 'cpu'

In [ ]:
# ========================================
# 加载数据集元数据并配置模型
# ========================================
#
# 📊 **步骤1：加载数据集元数据**
#
# 尝试从两个可能的路径加载数据集元数据
try:
    # 优先尝试 ./demo_data_language 目录
    dataset_metadata = LeRobotDatasetMetadata("omy_pnp_language", root='./demo_data_language')
except:
    # 如果失败，尝试 ./omy_pnp_language 目录
    dataset_metadata = LeRobotDatasetMetadata("omy_pnp_language", root='./omy_pnp_language')

# 💡 **什么是数据集元数据？**
# 包含数据集的重要信息：
# - features: 输入输出特征定义（图像、状态、动作等）
# - stats: 数据的统计信息（均值、标准差等，用于归一化）
# - info: 数据集基本信息（大小、版本等）

# ────────────────────────────────────────
# 📊 **步骤2：分离输入和输出特征**
# ────────────────────────────────────────

# 将数据集特征转换为策略可用的格式
features = dataset_to_policy_features(dataset_metadata.features)

# 输出特征：机器人要执行的动作
# FeatureType.ACTION = 动作类型（关节角度、夹爪状态等）
output_features = {key: ft for key, ft in features.items() if ft.type is FeatureType.ACTION}

# 输入特征：除动作外的所有观测
# 包括：图像、关节状态、任务描述等
input_features = {key: ft for key, ft in features.items() if key not in output_features}

print("✅ 输入特征:", list(input_features.keys()))
print("✅ 输出特征:", list(output_features.keys()))

# ────────────────────────────────────────
# 📊 **步骤3：创建SmolVLA模型配置**
# ────────────────────────────────────────

# SmolVLAConfig: 定义模型的架构和行为
cfg = SmolVLAConfig(
    input_features=input_features,   # 输入特征定义
    output_features=output_features,  # 输出特征定义
    
    # 🔑 **关键参数说明**
    chunk_size=5,       # 动作序列长度
    # 模型一次预测5步未来的动作
    # 例如：[step1, step2, step3, step4, step5]
    # 这样可以让动作更平滑、更有规划性
    
    n_action_steps=5,   # 实际执行的动作步数
    # 从预测的5步动作中，实际执行5步
    # 然后重新观测、重新预测（滚动预测）
)

# 💡 **为什么chunk_size和n_action_steps都是5？**
# - chunk_size=5: 每次预测5步动作
# - n_action_steps=5: 每次执行5步动作
# - 这样每次都执行完所有预测的动作，然后重新预测
# 
# 其他常见配置：
# - chunk_size=10, n_action_steps=5: 预测10步，执行5步（有重叠，更平滑）
# - chunk_size=1, n_action_steps=1: 每次只预测和执行1步（反应快，但可能抖动）

# ────────────────────────────────────────
# 📊 **步骤4：解析时间戳**
# ────────────────────────────────────────

# 计算时间戳差值（用于时序数据对齐）
delta_timestamps = resolve_delta_timestamps(cfg, dataset_metadata)

print("✅ 模型配置创建完成！")
print(f"   Chunk size: {cfg.chunk_size}")
print(f"   Action steps: {cfg.n_action_steps}")

In [ ]:
# ========================================
# 加载模型：最终checkpoint（已注释）
# ========================================
#
# 📝 **这个cell的作用**
# 加载训练完成后的最后一个checkpoint（last）
#
# 🔧 **使用方法**
# 1. 取消注释（删除开头的#）
# 2. 运行这个cell
#
# 📂 **模型路径说明**
# ckpt/smolvla_omy4060_20251103_224510/checkpoints/last/pretrained_model
#   │                                               │
#   └─ 训练时间戳                                   └─ 最后的checkpoint
#
# 💡 **什么是checkpoint？**
# - 训练过程中保存的模型快照
# - 包含模型权重、优化器状态等
# - 可以用于继续训练或直接部署
#
# 📊 **last vs 其他checkpoint**
# - last: 训练的最后一步（不一定是最好的）
# - 002000: 训练2000步时的checkpoint
# - 010000: 训练10000步时的checkpoint
# 
# ⚠️  **注意**
# - 只运行一个模型加载cell！
# - 如果要测试不同checkpoint，注释掉其他的

# # 从本地checkpoint加载模型
# policy = SmolVLAPolicy.from_pretrained(
#     'ckpt/smolvla_omy4060_20251103_224510/checkpoints/last/pretrained_model',
#     dataset_stats=dataset_metadata.stats  # 传入数据集统计信息（用于归一化）
# )

# # 或者从Hugging Face Hub加载预训练模型
# # policy = SmolVLAPolicy.from_pretrained(
# #     "Jeongeun/omy_pnp_pi0",  # Hugging Face模型ID
# #     config=cfg,              # 模型配置
# #     dataset_stats=dataset_metadata.stats
# # )

# # 将模型移到GPU
# policy.to(device)

# print("✅ 模型加载完成！")
# print(f"   设备: {device}")
# print(f"   模型参数量: {sum(p.numel() for p in policy.parameters()) / 1e6:.1f}M")

In [ ]:
# ========================================
# 加载模型：2000步checkpoint（已注释）
# ========================================
#
# 📝 **这个cell的作用**
# 加载训练2000步时的checkpoint，用于测试早期训练结果
#
# 🔬 **为什么测试2000步的模型？**
# - 观察模型在训练早期的表现
# - 对比不同训练阶段的性能差异
# - 判断模型是否需要继续训练
#
# 📊 **预期表现**
# - 2000步：模型刚开始学习，成功率可能较低
# - 可能只学会了基本动作，任务完成度不高
#
# 🔧 **使用方法**
# 1. 注释掉其他模型加载cell
# 2. 取消这个cell的注释
# 3. 运行这个cell
#
# 💡 **模型路径解释**
# ckpt/smolvla_omy4060_20251103_224510/checkpoints/002000/pretrained_model
#                                                     └─ 2000步checkpoint

# # 从2000步checkpoint加载模型
# policy = SmolVLAPolicy.from_pretrained(
#     'ckpt/smolvla_omy4060_20251103_224510/checkpoints/002000/pretrained_model',
#     dataset_stats=dataset_metadata.stats
# )

# # 将模型移到GPU
# policy.to(device)

# print("✅ 模型加载完成（2000步checkpoint）！")
# print(f"   训练步数: 2000")
# print(f"   设备: {device}")

In [ ]:
# ========================================
# 加载模型：10000步checkpoint（当前使用）
# ========================================
#
# 📝 **这个cell的作用**
# 加载训练10000步时的checkpoint，用于测试充分训练后的模型性能
#
# 🏆 **为什么选择10000步？**
# - 模型已经充分学习，性能接近最优
# - 比2000步的模型表现更好
# - 通常是最佳性能的checkpoint
#
# 📊 **预期表现**
# - 10000步：模型已经学会完整任务流程
# - 成功率应该明显高于2000步的模型
# - 动作更平滑、更可靠
#
# 💡 **模型路径解释**
# ckpt/smolvla_omy4060_20251103_224510/checkpoints/010000/pretrained_model
#   │                                             └─ 10000步checkpoint
#   └─ checkpoint根目录
#
# 🔄 **from_pretrained方法做了什么？**
# 1. 读取模型配置文件（config.json）
# 2. 加载模型权重（model.safetensors）
# 3. 加载数据集统计信息（用于输入输出归一化）
# 4. 初始化模型并设置为评估模式
#
# 🎯 **关键参数**
# - dataset_stats: 数据集统计信息（均值、标准差）
#   用于将观测值归一化到[-1, 1]或[0, 1]范围
#   这样模型才能正确处理输入

# ✅ 从10000步checkpoint加载模型（当前使用）
policy = SmolVLAPolicy.from_pretrained(
    'ckpt/smolvla_omy4060_20251103_224510/checkpoints/010000/pretrained_model',
    dataset_stats=dataset_metadata.stats
)

# 或者从Hugging Face Hub加载（备选方案）
# policy = SmolVLAPolicy.from_pretrained(
#     "Jeongeun/omy_pnp_pi0",
#     config=cfg,
#     dataset_stats=dataset_metadata.stats
# )

# 将模型移动到GPU进行推理
policy.to(device)

print("✅ 模型加载完成（10000步checkpoint）！")
print(f"   训练步数: 10000")
print(f"   设备: {device}")
print(f"   模型参数量: {sum(p.numel() for p in policy.parameters()) / 1e6:.1f}M")
print(f"   显存占用: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

In [ ]:
# ========================================
# 环境变量设置（重复设置，确保配置生效）
# ========================================
#
# ⚠️  **为什么再次设置环境变量？**
# 
# 在某些情况下，环境变量可能在运行过程中丢失或被覆盖
# 特别是在：
# - 重启kernel后
# - 加载某些库后（如mujoco）
# - 长时间运行后
#
# 💡 **最佳实践**
# 在创建MuJoCo环境之前，再次确认环境变量设置
# 这样可以确保仿真渲染正常工作
#
# 📝 **详细说明请参考第一个cell**

import os

# 1. 设置DISPLAY
os.environ['DISPLAY'] = ':0'
os.environ['XAUTHORITY'] = os.path.expanduser('~/.Xauthority')
print(f"✓ DISPLAY设置为: {os.environ['DISPLAY']}")

# 2. 强制使用GPU渲染(关键!)
os.environ['MUJOCO_GL'] = 'egl'  # EGL后端GPU加速
print(f"✓ MUJOCO_GL: egl (GPU硬件加速)")

# 3. NVIDIA GPU优化
os.environ['__GL_SYNC_TO_VBLANK'] = '0'  # 关闭垂直同步
os.environ['__GL_YIELD'] = 'NOTHING'      # 减少CPU等待
print("✓ NVIDIA GPU优化已启用")

# 4. OpenGL性能优化
os.environ['__GL_FSAA_MODE'] = '0'        # 关闭抗锯齿
os.environ['__GL_LOG_MAX_ANISO'] = '0'    # 关闭各向异性过滤
print("✓ OpenGL性能优化已启用")

print("\n✅ 环境变量重新设置完成！")

In [ ]:
# ========================================
# 创建MuJoCo仿真环境
# ========================================
#
# 🎮 **创建环境实例**
#
# SimpleEnv2: 自定义的MuJoCo环境类
# - 封装了MuJoCo物理引擎
# - 提供机器人控制接口
# - 支持摄像头图像获取
# - 实现任务成功判定逻辑
#
# 📂 **参数说明**
#
# xml_path: './asset/example_scene_y2.xml'
#   - MuJoCo场景描述文件（XML格式）
#   - 包含机器人模型、物体、摄像头、灯光等
#   - 定义了整个仿真世界的物理属性
#
# action_type: 'joint_angle'
#   - 控制模式：关节角度控制
#   - 其他可选模式：
#     * 'joint_velocity': 关节速度控制
#     * 'joint_torque': 关节力矩控制
#     * 'end_effector': 末端执行器位置控制
#
# 🤖 **环境包含什么？**
# - Parol6机械臂（6自由度）
# - 夹爪（1自由度，开合）
# - 桌面场景
# - 可抓取物体（杯子、盘子等）
# - 3个摄像头（固定视角、俯视、腕部）
# - 物理碰撞检测
#
# 🔄 **环境生命周期**
# 创建 → 重置(reset) → 观测(observe) → 动作(step) → 渲染(render) → 循环
#
# ⏱️  **初始化时间**
# - 第一次创建：1-3秒（加载模型、编译场景）
# - 后续重置：<0.1秒

from mujoco_env.y_env2 import SimpleEnv2

# XML场景文件路径
xml_path = './asset/example_scene_y2.xml'

# 创建环境实例
PnPEnv = SimpleEnv2(xml_path, action_type='joint_angle')

print("✅ MuJoCo环境创建成功！")
print(f"   场景文件: {xml_path}")
print(f"   控制模式: joint_angle")
print(f"   关节数量: {len(PnPEnv.joint_names)}")
print(f"   关节名称: {PnPEnv.joint_names}")
print(f"   任务描述: {PnPEnv.instruction}")

In [ ]:
# ========================================
# 定义图像预处理函数
# ========================================
#
# 🖼️  **为什么需要图像预处理？**
#
# MuJoCo渲染的图像格式：
# - NumPy数组
# - 形状：(H, W, 3)
# - 像素值范围：[0, 255]（uint8）
# - 颜色通道顺序：RGB
#
# 模型需要的图像格式：
# - PyTorch张量
# - 形状：(3, H, W)（通道优先）
# - 像素值范围：[0.0, 1.0]（float32）
# - 已归一化
#
# 🔄 **transforms.ToTensor() 做了什么？**
# 1. 将PIL图像或NumPy数组转换为PyTorch张量
# 2. 改变维度顺序：(H, W, C) → (C, H, W)
# 3. 归一化像素值：[0, 255] → [0.0, 1.0]
# 4. 转换数据类型：uint8 → float32
#
# 💡 **为什么不直接用NumPy数组？**
# - PyTorch模型需要Tensor输入
# - Tensor支持GPU加速
# - Tensor支持自动梯度计算
# - 归一化后的值更适合神经网络
#
# 📝 **使用示例**
# >>> img = np.random.randint(0, 256, (480, 640, 3), dtype=np.uint8)  # MuJoCo图像
# >>> transform = get_default_transform()
# >>> tensor_img = transform(Image.fromarray(img))  # PIL转换
# >>> print(tensor_img.shape)  # torch.Size([3, 480, 640])
# >>> print(tensor_img.min(), tensor_img.max())  # tensor(0.0) tensor(1.0)

from torchvision import transforms

def get_default_transform(image_size: int = 224):
    """
    获取默认的图像预处理变换
    
    Parameters:
    -----------
    image_size : int, default=224
        目标图像大小（这个参数在当前实现中未使用）
        如果需要调整大小，可以添加 transforms.Resize((image_size, image_size))
    
    Returns:
    --------
    transform : torchvision.transforms.Compose
        图像预处理变换序列
        
    Example:
    --------
    >>> transform = get_default_transform()
    >>> pil_image = Image.fromarray(numpy_array)
    >>> tensor_image = transform(pil_image)
    """
    return transforms.Compose([
        transforms.ToTensor(),  # PIL图像 [0–255] → FloatTensor [0.0–1.0]
                               # 形状变化: (H, W, C) → (C, H, W)
    ])

# 创建全局变量，供后续使用
IMG_TRANSFORM = get_default_transform()

print("✅ 图像预处理函数定义完成！")
print(f"   输入格式: PIL Image (H, W, 3), [0-255]")
print(f"   输出格式: Torch Tensor (3, H, W), [0.0-1.0]")

In [ ]:
# ========================================================================
# 🚀 SmolVLA策略评估脚本（带超时机制和成功率统计）
# SmolVLA Policy Evaluation with Timeout and Success Rate Statistics
# ========================================================================
#
# 📋 **这个cell的作用**
# 这是整个笔记本最核心的部分！
# 它会：
# 1. 运行100个测试回合（可配置）
# 2. 每个回合让模型控制机器人完成抓取和放置任务
# 3. 统计成功率、平均步数、平均耗时等指标
# 4. 保存评估结果到文件
#
# 🎯 **评估流程**
# ┌──────────────┐
# │ 开始评估循环  │
# └──────┬───────┘
#        ↓
# ┌──────────────────────────────────────────────┐
# │  for episode in range(NUM_EPISODES):        │
# │  ┌────────────────────────────────────┐     │
# │  │ 1. 重置环境和策略                   │     │
# │  │ 2. 获取观测（图像、关节状态）        │     │
# │  │ 3. 模型推理 → 预测动作              │     │
# │  │ 4. 执行动作                        │     │
# │  │ 5. 检查成功/超时                   │     │
# │  │ 6. 重复步骤2-5直到成功或超时        │     │
# │  └────────────────────────────────────┘     │
# └──────────────────────────────────────────────┘
#        ↓
# ┌──────────────┐
# │ 打印统计结果  │
# └──────────────┘
#
# ⏱️  **预计运行时间**
# - 100回合 × 平均15秒/回合 ≈ 25分钟
# - 可以修改NUM_EPISODES来调整测试数量
#
# 💾 **输出文件**
# evaluation_results_YYYYMMDD_HHMMSS.txt
# - 包含详细的统计信息
# - 保存在当前目录

import time
import numpy as np
from datetime import datetime
from collections import defaultdict

# ========================================================================
# ⚙️  配置参数 | Configuration
# ========================================================================

NUM_EPISODES = 100        # 测试回合总数 | Total number of episodes to test
                         # 💡 初学者建议：先设置为10，快速验证流程
                         # 💡 正式评估：设置为100或更多
                         
TIMEOUT_SECONDS = 30      # 每回合超时时间（秒）| Timeout for each episode
                         # 💡 如果机器人30秒内没有完成任务，视为失败
                         
CONTROL_FREQUENCY = 20    # 控制频率（赫兹）| Control frequency (Hz)
                         # 💡 每秒执行20次控制（20Hz = 0.05秒/次）
                         # 💡 更高频率 = 更平滑的控制，但计算量更大
                         
MAX_STEPS_PER_EPISODE = TIMEOUT_SECONDS * CONTROL_FREQUENCY  
# 最大步数 = 30秒 × 20Hz = 600步

# ========================================================================
# 📊 统计数据容器 | Statistics Container
# ========================================================================

class EvaluationStats:
    """
    评估统计数据容器
    Container for evaluation statistics
    
    用途：
    - 记录每个回合的结果
    - 计算成功率、平均值等统计指标
    - 生成评估报告
    """
    def __init__(self):
        self.total_episodes = 0          # 总回合数
        self.successful_episodes = 0     # 成功回合数
        self.failed_episodes = 0         # 失败回合数
        self.timeout_episodes = 0        # 超时回合数
        self.episode_durations = []      # 每回合耗时列表（秒）
        self.episode_steps = []          # 每回合步数列表
        self.success_steps = []          # 成功回合的步数列表
        
    def add_episode(self, success, timeout, duration, steps):
        """
        添加回合结果到统计数据
        Add episode result to statistics
        
        Args:
            success (bool): 是否成功 | Whether the episode succeeded
            timeout (bool): 是否超时 | Whether the episode timed out
            duration (float): 回合耗时（秒）| Episode duration in seconds
            steps (int): 执行的步数 | Number of steps taken
        """
        self.total_episodes += 1
        self.episode_durations.append(duration)
        self.episode_steps.append(steps)
        
        if success:
            self.successful_episodes += 1
            self.success_steps.append(steps)
        elif timeout:
            self.timeout_episodes += 1
            self.failed_episodes += 1
        else:
            self.failed_episodes += 1
    
    def get_success_rate(self):
        """计算成功率 | Calculate success rate"""
        if self.total_episodes == 0:
            return 0.0
        return (self.successful_episodes / self.total_episodes) * 100
    
    def print_summary(self):
        """
        打印评估总结
        Print evaluation summary
        """
        print("\n" + "="*80)
        print("📊 EVALUATION SUMMARY | 评估总结")
        print("="*80)
        
        print(f"\n📈 Overall Statistics | 总体统计:")
        print(f"  Total Episodes | 总回合数:        {self.total_episodes}")
        print(f"  Successful | 成功:               {self.successful_episodes}")
        print(f"  Failed | 失败:                   {self.failed_episodes}")
        print(f"  Timeout | 超时:                  {self.timeout_episodes}")
        print(f"  Success Rate | 成功率:           {self.get_success_rate():.2f}%")
        
        if self.episode_durations:
            print(f"\n⏱️  Time Statistics | 时间统计:")
            print(f"  Average Duration | 平均耗时:    {np.mean(self.episode_durations):.2f}s")
            print(f"  Min Duration | 最短耗时:        {np.min(self.episode_durations):.2f}s")
            print(f"  Max Duration | 最长耗时:        {np.max(self.episode_durations):.2f}s")
        
        if self.episode_steps:
            print(f"\n🎯 Step Statistics | 步数统计:")
            print(f"  Average Steps | 平均步数:       {np.mean(self.episode_steps):.1f}")
            print(f"  Min Steps | 最少步数:           {np.min(self.episode_steps)}")
            print(f"  Max Steps | 最多步数:           {np.max(self.episode_steps)}")
        
        if self.success_steps:
            print(f"\n✅ Success Statistics | 成功回合统计:")
            print(f"  Avg Steps (Success) | 成功平均步数: {np.mean(self.success_steps):.1f}")
            print(f"  Min Steps (Success) | 成功最少步数: {np.min(self.success_steps)}")
            print(f"  Max Steps (Success) | 成功最多步数: {np.max(self.success_steps)}")
        
        print("="*80 + "\n")

# ========================================================================
# 🎯 主评估函数 | Main Evaluation Loop
# ========================================================================

def evaluate_policy_with_timeout():
    """
    评估策略性能（带超时机制）
    Evaluate policy performance with timeout mechanism
    
    工作流程：
    1. 初始化统计对象
    2. 循环NUM_EPISODES次
    3. 每次循环：
       - 重置环境
       - 获取观测
       - 模型推理
       - 执行动作
       - 检查成功/超时
    4. 打印和保存统计结果
    """
    # ═══════════════════════════════════════════════════════════════
    # 初始化
    # ═══════════════════════════════════════════════════════════════
    stats = EvaluationStats()
    evaluation_start_time = datetime.now()
    
    print("="*80)
    print("🚀 STARTING POLICY EVALUATION | 开始策略评估")
    print("="*80)
    print(f"Start Time | 开始时间: {evaluation_start_time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Total Episodes | 总回合数: {NUM_EPISODES}")
    print(f"Timeout | 超时时间: {TIMEOUT_SECONDS}s")
    print(f"Control Frequency | 控制频率: {CONTROL_FREQUENCY}Hz")
    print("="*80 + "\n")
    
    # ═══════════════════════════════════════════════════════════════
    # 回合循环 | Episode Loop
    # ═══════════════════════════════════════════════════════════════
    for episode_idx in range(NUM_EPISODES):
        # ───────────────────────────────────────────────────────────
        # 📌 步骤1：重置环境和策略
        # ───────────────────────────────────────────────────────────
        PnPEnv.reset(seed=episode_idx)  # 使用不同seed确保场景多样性
        policy.reset()                   # 重置策略内部状态
        policy.eval()                    # 设置为评估模式（关闭dropout等）
        
        # 初始化回合变量
        step = 0                         # 当前步数计数器
        episode_start_time = time.time() # 记录回合开始时间
        episode_success = False          # 成功标志
        episode_timeout = False          # 超时标志
        
        print(f"Episode {episode_idx + 1}/{NUM_EPISODES} started... | 回合 {episode_idx + 1}/{NUM_EPISODES} 开始...", end=" ")
        
        # ───────────────────────────────────────────────────────────
        # 🔄 步骤循环 | Step Loop
        # ───────────────────────────────────────────────────────────
        while PnPEnv.env.is_viewer_alive():
            # 推进物理仿真（每帧都执行）
            PnPEnv.step_env()
            
            # 按控制频率执行（20Hz = 每秒20次）
            if PnPEnv.env.loop_every(HZ=CONTROL_FREQUENCY):
                
                # ═══════════════════════════════════════════════════
                # ⏱️  超时检查 | Timeout Check
                # ═══════════════════════════════════════════════════
                current_time = time.time()
                elapsed_time = current_time - episode_start_time
                
                if elapsed_time > TIMEOUT_SECONDS:
                    episode_timeout = True
                    print(f"⏱️  TIMEOUT after {elapsed_time:.1f}s | 超时")
                    break
                
                # ═══════════════════════════════════════════════════
                # ✅ 成功检查 | Success Check
                # ═══════════════════════════════════════════════════
                success = PnPEnv.check_success()
                if success:
                    episode_success = True
                    episode_duration = time.time() - episode_start_time
                    print(f"✅ SUCCESS in {episode_duration:.2f}s ({step} steps) | 成功")
                    break
                
                # ═══════════════════════════════════════════════════
                # 📷 获取观测 | Get Observation
                # ═══════════════════════════════════════════════════
                
                # 1️⃣ 获取机器人关节状态（前6个关节）
                state = PnPEnv.get_joint_state()[:6]
                # state是一个长度为6的数组，包含6个关节的当前角度
                # 例如：[0.1, -0.5, 1.2, 0.0, 0.8, -0.3]（单位：弧度）
                
                # 2️⃣ 获取两个摄像头的图像
                image, wrist_image = PnPEnv.grab_image()
                # image: 第三人称视角（固定摄像头）
                # wrist_image: 第一人称视角（腕部摄像头）
                
                # 3️⃣ 图像预处理
                # NumPy数组 → PIL图像 → 调整大小 → PyTorch张量
                image = Image.fromarray(image)           # (H,W,3) uint8 → PIL
                image = image.resize((256, 256))         # 调整到256x256
                image = IMG_TRANSFORM(image)             # PIL → Tensor (3,256,256) [0,1]
                
                wrist_image = Image.fromarray(wrist_image)
                wrist_image = wrist_image.resize((256, 256))
                wrist_image = IMG_TRANSFORM(wrist_image)
                
                # ═══════════════════════════════════════════════════
                # 🎯 准备模型输入 | Prepare Model Input
                # ═══════════════════════════════════════════════════
                data = {
                    # 关节状态：[batch=1, joints=6]
                    'observation.state': torch.tensor([state]).to(device),
                    
                    # 第三人称图像：[batch=1, channels=3, height=256, width=256]
                    'observation.image': image.unsqueeze(0).to(device),
                    
                    # 腕部图像：[batch=1, channels=3, height=256, width=256]
                    'observation.wrist_image': wrist_image.unsqueeze(0).to(device),
                    
                    # 任务描述（自然语言）：列表，包含1个字符串
                    'task': [PnPEnv.instruction],  
                    # 例如："pick the mug and place it in the target area"
                }
                
                # ═══════════════════════════════════════════════════
                # 🧠 策略推理 | Policy Inference
                # ═══════════════════════════════════════════════════
                with torch.no_grad():  # 关闭梯度计算（推理时不需要）
                    # 模型预测动作序列
                    action = policy.select_action(data)
                    # 输出形状: [batch=1, chunk_size=5, action_dim=7]
                    # 取第一个batch的第一步动作
                    action = action[0, :7].cpu().numpy()  # [7]
                
                # 💡 动作维度说明：
                # action[0:6] - 6个关节的目标角度（弧度）
                # action[6]   - 夹爪开合状态（-1=全开，1=全闭）
                
                # ═══════════════════════════════════════════════════
                # 🎮 执行动作 | Execute Action
                # ═══════════════════════════════════════════════════
                _ = PnPEnv.step(action)
                # step函数内部会：
                # 1. 将动作发送给MuJoCo控制器
                # 2. 执行物理仿真
                # 3. 更新机器人状态
                
                # ═══════════════════════════════════════════════════
                # 🖼️  渲染可视化 | Render Visualization
                # ═══════════════════════════════════════════════════
                PnPEnv.render()
                # 更新仿真窗口显示
                
                # 步数递增
                step += 1
                
                # ═══════════════════════════════════════════════════
                # 🛡️  步数限制检查 | Step Limit Check
                # ═══════════════════════════════════════════════════
                if step >= MAX_STEPS_PER_EPISODE:
                    episode_timeout = True
                    print(f"⏱️  MAX STEPS REACHED | 达到最大步数")
                    break
        
        # ───────────────────────────────────────────────────────────
        # 📊 回合结束：记录统计数据
        # ───────────────────────────────────────────────────────────
        episode_duration = time.time() - episode_start_time
        
        stats.add_episode(
            success=episode_success,
            timeout=episode_timeout,
            duration=episode_duration,
            steps=step
        )
        
        # 每10回合打印进度
        if (episode_idx + 1) % 10 == 0:
            current_success_rate = stats.get_success_rate()
            print(f"\n📊 Progress: {episode_idx + 1}/{NUM_EPISODES} | "
                  f"Success Rate: {current_success_rate:.2f}% | "
                  f"成功率: {current_success_rate:.2f}%\n")
    
    # ═══════════════════════════════════════════════════════════════
    # 🎉 评估完成 | Evaluation Complete
    # ═══════════════════════════════════════════════════════════════
    evaluation_end_time = datetime.now()
    total_duration = (evaluation_end_time - evaluation_start_time).total_seconds()
    
    print("\n" + "="*80)
    print("🎉 EVALUATION COMPLETED | 评估完成")
    print("="*80)
    print(f"End Time | 结束时间: {evaluation_end_time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Total Duration | 总耗时: {total_duration:.2f}s ({total_duration/60:.2f} minutes)")
    print("="*80)
    
    # 打印详细统计
    stats.print_summary()
    
    # 保存结果到文件
    save_evaluation_results(stats, evaluation_start_time, evaluation_end_time)
    
    return stats

# ========================================================================
# 💾 保存结果到文件 | Save Results to File
# ========================================================================

def save_evaluation_results(stats, start_time, end_time):
    """
    保存评估结果到文本文件
    Save evaluation results to a text file
    
    Args:
        stats: EvaluationStats对象
        start_time: 评估开始时间
        end_time: 评估结束时间
    """
    timestamp = start_time.strftime("%Y%m%d_%H%M%S")
    filename = f"evaluation_results_{timestamp}.txt"
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write("="*80 + "\n")
        f.write("SmolVLA Policy Evaluation Results\n")
        f.write("SmolVLA策略评估结果\n")
        f.write("="*80 + "\n\n")
        
        f.write(f"Start Time | 开始时间: {start_time.strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"End Time | 结束时间: {end_time.strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Duration | 总耗时: {(end_time - start_time).total_seconds():.2f}s\n\n")
        
        f.write(f"Total Episodes | 总回合数: {stats.total_episodes}\n")
        f.write(f"Successful | 成功: {stats.successful_episodes}\n")
        f.write(f"Failed | 失败: {stats.failed_episodes}\n")
        f.write(f"Timeout | 超时: {stats.timeout_episodes}\n")
        f.write(f"Success Rate | 成功率: {stats.get_success_rate():.2f}%\n\n")
        
        if stats.episode_durations:
            f.write(f"Average Duration | 平均耗时: {np.mean(stats.episode_durations):.2f}s\n")
            f.write(f"Average Steps | 平均步数: {np.mean(stats.episode_steps):.1f}\n")
        
        if stats.success_steps:
            f.write(f"Average Steps (Success) | 成功平均步数: {np.mean(stats.success_steps):.1f}\n")
    
    print(f"📝 Results saved to: {filename} | 结果已保存到: {filename}")

# ========================================================================
# ▶️  执行评估 | Execute Evaluation
# ========================================================================

if __name__ == "__main__":
    # 💡 提示：第一次运行建议先设置NUM_EPISODES=10测试
    # 💡 Tip: For first run, suggest setting NUM_EPISODES=10 for testing
    evaluation_stats = evaluate_policy_with_timeout()

In [ ]:
# ========================================
# 上传模型到Hugging Face Hub（可选，已注释）
# ========================================
#
# 📤 **这个cell的作用**
# 将训练好的模型上传到Hugging Face Hub，方便分享和部署
#
# 🌐 **Hugging Face Hub是什么？**
# - 类似GitHub，但专门用于机器学习模型
# - 可以托管、分享、下载预训练模型
# - 支持版本控制和协作
#
# 🔧 **使用方法**
# 1. 在Hugging Face网站注册账号（https://huggingface.co）
# 2. 获取访问令牌（Access Token）
# 3. 登录：huggingface-cli login
# 4. 取消下面代码的注释
# 5. 修改repo_id为你自己的仓库ID
# 6. 运行这个cell
#
# 💡 **参数说明**
# repo_id: 'Jeongeun/omy_pnp_smolvla'
#   - 格式：'用户名/仓库名'
#   - 需要修改为你自己的用户名
#
# commit_message: 提交信息
#   - 描述这次上传的内容
#   - 类似git commit message
#
# ⚠️  **注意**
# - 需要先登录Hugging Face CLI
# - 上传大型模型需要时间（几GB可能需要几分钟）
# - 确保你有权限push到指定的仓库
#
# 📚 **登录命令**
# !huggingface-cli login
#
# 🔗 **上传后的用途**
# - 其他人可以直接下载你的模型
# - 使用方法：policy = SmolVLAPolicy.from_pretrained('你的用户名/仓库名')

# policy.push_to_hub(
#     repo_id='Jeongeun/omy_pnp_smolvla',  # 🔴 修改为你自己的仓库ID
#     commit_message='Add trained policy for PnP task',
# )

# print("✅ 模型已上传到Hugging Face Hub！")
# print(f"   访问链接：https://huggingface.co/你的用户名/仓库名")

In [ ]:
# ========================================
# 关闭MuJoCo仿真窗口
# ========================================
#
# 🛑 **这个cell的作用**
# 关闭MuJoCo的可视化窗口，释放资源
#
# ⚠️  **重要提示**
# - 运行这个cell后，仿真窗口会关闭
# - 如果要重新运行评估，需要重新创建环境（运行创建环境的cell）
# - 关闭窗口不会删除已保存的评估结果文件
#
# 💡 **什么时候运行这个cell？**
# - 评估完成后
# - 不需要再看仿真画面时
# - 准备关闭notebook或切换到其他任务时
#
# 🔄 **如果要重新开始**
# 1. 运行这个cell关闭当前环境
# 2. 重新运行"创建MuJoCo仿真环境"cell
# 3. 重新运行评估cell
#
# 📝 **底层操作**
# PnPEnv.env.close_viewer() 会：
# - 关闭OpenGL窗口
# - 释放GPU资源
# - 清理MuJoCo上下文

PnPEnv.env.close_viewer()

print("✅ MuJoCo仿真窗口已关闭")
print("💡 如需重新运行评估，请重新创建环境")